# Initialize Simulation

In [1]:
from myAgent import CarlaAgent

In [2]:
myAgent = CarlaAgent()
myAgent.spawn_vehicle()
myAgent.attach_camera()
myAgent.attach_cameraS()
# myAgent.autopilot(20)
myAgent.attach_controller()
myAgent.find_vehicle()
print("deployed")

Establishing Connection to Server
created vehicle.tesla.model3
created sensor.camera.rgb
created sensor.camera.semantic_segmentation
deployed


# Perception Test

In [3]:
from perception.tools.ddrnetSegmentation import DDRNet
from perception.tools.yoloDetector import Yolo

import cv2
import matplotlib.pyplot as plt
import numpy as np
import time

segmenter = DDRNet()
detector = Yolo()

Using torch 1.8.0 CUDA:0 (GeForce RTX 3090, 24260MB)



Segmentation Model Initialized!
Fusing layers... 


Model Summary: 484 layers, 88922205 parameters, 0 gradients


Detector Model Initialized!


### Simulation

In [4]:
sim_time = 120

print("Demo driving test")

# Drive straight
# myAgent.control.throttle = 0
# myAgent.control.brake = 0
# myAgent.control.steer = 0
# myAgent.control.hand_brake = 0
# myAgent.control.reverse = 0

# myAgent.vehicle.apply_control(myAgent.control)


IMAGE_H, IMAGE_W, _ = myAgent.image.shape
src = np.float32([[0, IMAGE_H], [IMAGE_W, IMAGE_H],
                  [int(2*IMAGE_W/5), int(IMAGE_H/2)], [int(3*IMAGE_W/5), int(IMAGE_H/2)]])
dst = np.float32([[300, IMAGE_H], [500, IMAGE_H], [300, 0], [500, 0]])
M = cv2.getPerspectiveTransform(src, dst) # The transformation matrix

h = int(IMAGE_H/2)
w = int(IMAGE_W/2)
    
myAgent.vehicle.set_autopilot(True)

t = time.time()
while(time.time()-t < sim_time):
    
    road_class = 7
    RGB = np.copy(myAgent.image)
    sem = np.copy(myAgent.imageS[:,:,2])
    sem[sem!=road_class] = 0
    sem[sem==road_class] = 255
    
    segmented_image = segmenter.segments(RGB)
    
    threshold = 4
    segseg = segmented_image[0].cpu().detach().numpy()
    segseg = np.copy(segseg[0, :, :, :])
    segseg = segseg.transpose((1, 2, 0))
    segseg = segseg[:, :, 0]
#     segseg[segseg<=threshold] = 0
#     segseg[segseg>threshold] = 255
    
    detected_image = detector.detect(RGB)

    warped_RGB = cv2.warpPerspective(RGB, M, (IMAGE_W, IMAGE_H)) # Image warping
    warped_sem = cv2.warpPerspective(sem, M, (IMAGE_W, IMAGE_H)) # Image warping
    
    moment = cv2.moments(warped_sem[int(IMAGE_H/2):,int(IMAGE_W/4):int(3*IMAGE_W/4)])
    # calculate x,y coordinate of center
    try:
        cX = int(moment["m10"] / moment["m00"])+int(IMAGE_W/4)
        cY = int(moment["m01"] / moment["m00"])+int(IMAGE_H/4)
    except:
        cX = int(IMAGE_W/2)
        cY = int(IMAGE_H/2)
    # put text and highlight the center
    cv2.circle(warped_RGB, (cX, cY), 20, (0, 0, 255), -1)
    cv2.putText(warped_RGB, "centroid", (cX - 25, cY - 25),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

    
#     myAgent.control.steer = (cX-IMAGE_W/2)/(IMAGE_W)
#     myAgent.vehicle.apply_control(myAgent.control)

    
    cv2.imshow('Original Frame', cv2.resize(RGB, (w, h)))
    cv2.imshow('Original Warped', cv2.resize(warped_RGB, (w, h)))
    cv2.imshow('Segmentation Frame', cv2.resize(sem, (w, h)))
    cv2.imshow('Segmentation Warped', cv2.resize(warped_sem, (w, h)))
    cv2.imshow('Inferred Segmentation', cv2.resize(segseg, (w, h)))
    cv2.imshow('Object Detection', cv2.resize(detected_image, (w, h)))    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

myAgent.vehicle.set_autopilot(False)
# Stop
# myAgent.control.throttle = 0
# myAgent.control.brake = 0
# myAgent.control.steer = 0
# myAgent.control.hand_brake = 1
# myAgent.control.reverse = 0
# myAgent.vehicle.apply_control(myAgent.control)
cv2.destroyAllWindows()

print("done")

Demo driving test


/home/mahsay/anaconda3/envs/agent0/lib/python3.7/site-packages/torch/nn/functional.py:3455: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode)


done


### Reverse Gear

In [5]:
myAgent.control.throttle = 0.5
myAgent.control.brake = 0
myAgent.control.steer = 0
myAgent.control.hand_brake = 0
myAgent.control.reverse = 1

myAgent.vehicle.apply_control(myAgent.control)


t = time.time()
while(time.time()-t < 5):
    None
    
myAgent.control.throttle = 0
myAgent.control.brake = 0
myAgent.control.steer = 0
myAgent.control.hand_brake = 1
myAgent.control.reverse = 0

myAgent.vehicle.apply_control(myAgent.control)

# Terminate Simulation

In [6]:
myAgent.terminate()

destroying actors
terminated
